# Episodic Strategy를 사용한 LangGraph와 AgentCore Memory

## 소개

이 노트북은 LangGraph 프레임워크를 사용하여 대화형 AI Agent에 **episodic memory strategy**를 적용한 Amazon Bedrock AgentCore Memory를 통합하는 방법을 보여줍니다. 완전한 대화 세션을 캡처하는 episodic strategy에 초점을 맞추어, Agent가 특정 식사 계획 에피소드를 회상하고 식습관 패턴이 시간에 따라 어떻게 변화하는지 추적할 수 있도록 합니다.

## 튜토리얼 세부 정보

| 정보                | 세부 사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 장기 대화형                                                        |
| Agent 사용 사례     | Episodic Memory Strategy를 사용한 영양 어시스턴트                               |
| Agentic Framework   | LangGraph                                                                        |
| LLM model           | Anthropic Claude Sonnet 3.7                                                     |
| 튜토리얼 구성 요소  | AgentCore Memory, Episodic Strategy, LangGraph Hooks, 세션 기반 에피소드  |
| 예제 복잡도         | 중급                                                                     |

학습 내용:
- episodic memory strategy를 사용한 AgentCore Memory 생성
- 자동 메모리 저장을 위한 pre/post model hook 구현
- 식사 계획 세션을 기억하는 영양 어시스턴트 구축
- 과거 대화 검색 및 반영
- 시간에 따른 식습관 패턴 추적

### 시나리오 컨텍스트

이 예제에서는 episodic memory strategy를 사용하여 완전한 식사 계획 세션을 기억하는 **영양 어시스턴트**를 만듭니다. Agent는 레시피 논의, 재료 대체, 식사 피드백을 포함한 전체 대화 에피소드를 캡처합니다. 이를 통해 "지난주에 무엇을 계획했나요?"와 같은 시간적 쿼리와 식습관 패턴 분석이 가능합니다.

## 아키텍처

<div style="text-align:left">
    <img src="architecture_episodic.png" width="65%" />
</div>

### 영양 관리에 Episodic Memory Strategy를 사용하는 이유는?

- **세션 기반**: 각 식사 계획 대화가 하나의 에피소드
- **시간적 컨텍스트**: 식사가 특정 시간/상황과 연결됨
- **패턴 학습**: 선호도가 어떻게 변화하는지 추적
- **풍부한 회상**: 과거 추천의 전체 컨텍스트 기억

### Episodic Memory Strategy 작동 방식

episodic strategy는 상호작용을 구조화된 에피소드로 캡처하고 이러한 에피소드를 반영하여 의미 있는 인사이트를 생성하도록 설계되었습니다. 이 전략은 무슨 일이 일어났는지뿐만 아니라 각 에피소드의 의도, 생각, 결과도 기록합니다.

#### Episodic Strategy의 세 단계:

1. **Extraction** – 단기 메모리에서 유용한 인사이트를 식별하여 메모리 레코드로 장기 메모리에 배치
2. **Consolidation** – 유용한 정보를 새 레코드에 쓸지 기존 레코드에 쓸지 결정
3. **Reflection** – Agent 상호작용에서 에피소드 전반에 걸쳐 인사이트 생성

#### Strategy 출력:

**Episodes** (XML 형식):
- 상황, 의도, 평가, 정당화, 에피소드 수준 반영으로 세분화
- 상호작용이 진행됨에 따라 턴별로 분석
- 작업 순서 및 Tool 사용 이해에 도움

**Reflections** (백그라운드에서 생성):
- 여러 에피소드에 걸쳐 통합
- 다음을 식별하는 광범위한 인사이트 추출:
  - 성공적인 전략 및 패턴
  - 잠재적 개선 사항
  - 일반적인 실패 모드
  - 여러 상호작용에 걸쳐 학습한 교훈

#### 영양 어시스턴트의 경우:

- **Episodes**: 각 식사 계획 세션 (논의된 레시피, 재료, 결정)
- **Reflections**: 식습관 패턴, 선호하는 요리, 요리 기술 진행 상황
- **Turn-by-turn**: 레시피 탐색 → 재료 질문 → 대체 → 최종 선택

## 사전 요구 사항

- Python 3.10+
- 적절한 권한이 있는 AWS 계정
- AgentCore Memory에 대한 적절한 권한이 있는 AWS IAM 역할
- Amazon Bedrock model에 대한 액세스

환경 설정을 시작해 봅시다!


In [ ]:
# Install necessary libraries from https://github.com/langchain-ai/langchain-aws
%pip install -qr requirements.txt

In [ ]:
import os
import logging

# LangGraph와 LangChain 컴포넌트 import
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent  # ReAct 패턴 기반 agent 생성 함수
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnableConfig
from langgraph.store.base import BaseStore
import uuid


region = os.getenv("AWS_REGION", "us-east-1")
logging.getLogger("nutrition-agent").setLevel(logging.DEBUG)

In [ ]:
# AgentCore Memory를 store로 사용하기 위한 import
from langgraph_checkpoint_aws import AgentCoreMemoryStore

# 예제에서는 InMemorySaver를 사용하지만, 프로덕션에서는 AgentCoreMemorySaver 권장
# from langgraph_checkpoint_aws import AgentCoreMemorySaver
from langgraph.checkpoint.memory import InMemorySaver  # 대화 상태를 메모리에 임시 저장
from bedrock_agentcore.memory import MemoryClient

In [ ]:
import boto3
import json

# Memory 실행을 위한 IAM role 생성
iam_client = boto3.client("iam")
sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()["Account"]

ROLE_NAME = "AgentCoreMemoryExecutionRole"

# AgentCore Memory가 role을 assume할 수 있도록 trust policy 설정
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": [
                    "preprod.genesis-service.aws.internal",
                    "bedrock-agentcore.amazonaws.com",
                    "developer.genesis-service.aws.internal",
                ]
            },
            "Action": "sts:AssumeRole",
        }
    ],
}

# Bedrock model 호출 권한 부여
permissions_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": ["bedrock:InvokeModel", "bedrock:InvokeModelWithResponseStream"],
            "Resource": [
                "arn:aws:bedrock:*::foundation-model/*",
                "arn:aws:bedrock:*:*:inference-profile/*",
            ],
        }
    ],
}

try:
    # 기존 role이 있으면 재사용
    role = iam_client.get_role(RoleName=ROLE_NAME)
    MEMORY_EXECUTION_ROLE_ARN = role["Role"]["Arn"]
    print(f"✅ Using existing role: {MEMORY_EXECUTION_ROLE_ARN}")
except iam_client.exceptions.NoSuchEntityException:
    # Role이 없으면 새로 생성
    print(f"Creating IAM role: {ROLE_NAME}")
    role = iam_client.create_role(
        RoleName=ROLE_NAME,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description="Execution role for AgentCore Memory with custom strategies",
    )
    MEMORY_EXECUTION_ROLE_ARN = role["Role"]["Arn"]

    # Inline policy 연결
    iam_client.put_role_policy(
        RoleName=ROLE_NAME,
        PolicyName="BedrockModelAccess",
        PolicyDocument=json.dumps(permissions_policy),
    )
    print(f"✅ Created role: {MEMORY_EXECUTION_ROLE_ARN}")
    print("⏳ Waiting 10 seconds for IAM propagation...")
    import time

    time.sleep(10)  # IAM 변경사항이 전파될 때까지 대기

print(f"\nRole ARN: {MEMORY_EXECUTION_ROLE_ARN}")

In [ ]:
memory_name = "NutritionAssistantEpisodic"
client = MemoryClient(region_name=region)
MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Episodic strategy 커스터마이징: extraction, consolidation, reflection 단계별 설정
override_strategy = {
    "customMemoryStrategy": {
        "name": "NutritionEpisodicExtractor",
        "description": "Nutrition assistant with episodic memory for meal planning insights",
        "namespaces": ["nutrition/{actorId}/{sessionId}/"],  # 사용자별, 세션별로 메모리 구분
        "configuration": {
            "episodicOverride": {
                "extraction": {  # 대화에서 유용한 정보 추출
                    "modelId": MODEL_ID,
                    "appendToPrompt": "Extract meal planning conversations including recipes discussed, ingredients mentioned, dietary considerations, and user feedback.",
                },
                "consolidation": {  # 추출된 정보를 에피소드로 통합
                    "modelId": MODEL_ID,
                    "appendToPrompt": "Consolidate meal planning sessions into episodes, capturing the flow of recipe exploration and decision-making.",
                },
                "reflection": {  # 여러 에피소드에서 패턴과 인사이트 생성
                    "modelId": MODEL_ID,
                    "appendToPrompt": "Generate insights about dietary patterns, favorite recipes, and how meal preferences evolve over time.",
                    "namespaces": ["nutrition/{actorId}/"],  # Reflection은 세션 구분 없이 사용자별로만 저장
                },
            }
        },
    }
}

# Memory 생성 또는 기존 memory 가져오기
memory = client.create_or_get_memory(
    name=memory_name,
    description="Nutrition assistant with episodic memory for meal planning sessions",
    memory_execution_role_arn=MEMORY_EXECUTION_ROLE_ARN,
    strategies=[override_strategy],
)
memory_id = memory["id"]

print(f"✅ Created episodic memory: {memory_id}")

### Memory 구성 개요

AgentCore Episodic Memory 설정에는 다음이 포함됩니다:

- **Extraction**: 레시피, 재료, 피드백이 포함된 식사 계획 대화 캡처
- **Consolidation**: 대화를 식사 계획 에피소드로 그룹화
- **Reflection**: 시간에 따른 식습관 패턴 및 선호도에 대한 인사이트 생성
- **Namespaces**: 사용자별로 에피소드 구성 (`nutrition/{actorId}/`)

각 대화 세션은 회상하고 분석할 수 있는 에피소드가 됩니다.

## Step 3: Memory Store 및 LLM 초기화

이제 AgentCore Memory Store와 언어 모델을 초기화합니다.

In [ ]:
# 장기 메모리 저장 및 검색을 위한 store 초기화
store = AgentCoreMemoryStore(memory_id=memory_id, region_name=region)

# Bedrock LLM 초기화
llm = init_chat_model(MODEL_ID, model_provider="bedrock_converse", region_name=region)

## Step 4: Memory Hook 구현

메모리 저장을 자동으로 처리하기 위해 pre 및 post model hook을 생성합니다:

- **Pre-model hook**: LLM 호출 전에 사용자 메시지 저장
- **Post-model hook**: LLM 호출 후에 어시스턴트 응답 저장

### Memory 처리 작동 방식

1. 메시지가 actor_id 및 session_id와 함께 AgentCore Memory에 저장됨
2. episodic strategy가 대화를 처리하여 구조화된 에피소드 생성
3. 에피소드가 턴별 분석과 함께 `nutrition/{actorId}/{sessionId}/` namespace에 저장됨
4. Reflection이 에피소드 전반에 걸쳐 생성되고 `nutrition/{actorId}/` namespace에 저장됨
5. 각 에피소드는 상황, 의도, 평가, 대화 흐름을 캡처

**참고**: LangChain 메시지 타입은 store에 의해 내부적으로 AgentCore Memory 메시지 타입으로 변환되어 에피소드 및 reflection으로 적절하게 처리될 수 있습니다.


In [ ]:
def pre_model_hook(state, config: RunnableConfig, *, store: BaseStore):
    """LLM 호출 전에 실행되어 사용자 메시지를 저장하는 hook"""
    actor_id = config["configurable"]["actor_id"]
    thread_id = config["configurable"]["thread_id"]
    # Runtime에 받은 actor와 session 조합으로 메시지 저장
    namespace = (actor_id, thread_id)

    messages = state.get("messages", [])
    # LLM 호출 전 마지막 사용자 메시지 저장
    for msg in reversed(messages):
        if isinstance(msg, HumanMessage):
            store.put(namespace, str(uuid.uuid4()), {"message": msg})
            break

    # Episodic strategy는 메시지만 저장하면 됨 - 검색 불필요
    # Episode와 reflection은 백그라운드에서 자동 생성됨
    return {"messages": messages}


def post_model_hook(state, config: RunnableConfig, *, store: BaseStore):
    """LLM 호출 후에 실행되어 assistant 응답을 저장하는 hook"""
    actor_id = config["configurable"]["actor_id"]
    thread_id = config["configurable"]["thread_id"]

    # Runtime에 받은 actor와 session 조합으로 메시지 저장
    namespace = (actor_id, thread_id)

    messages = state.get("messages", [])
    # LLM 응답을 AgentCore Memory에 저장
    for msg in reversed(messages):
        if isinstance(msg, AIMessage):
            store.put(namespace, str(uuid.uuid4()), {"message": msg})
            break

    return {"messages": messages}

## Step 5: LangGraph Agent 생성

이제 메모리 hook이 통합된 LangGraph의 `create_react_agent`를 사용하여 영양 어시스턴트 Agent를 생성합니다. Tool 노드에는 장기 메모리 검색 Tool만 포함되며 pre 및 post model hook이 인수로 지정됩니다.

**참고**: 커스텀 Agent 구현의 경우 Store 및 Tool을 이 패턴에 따라 모든 워크플로우에 필요한 대로 구성할 수 있습니다. Pre/post model hook을 사용하거나, 전체 대화를 마지막에 저장하는 등의 방법이 있습니다.

In [ ]:
graph = create_react_agent(
    llm,
    store=store,
    tools=[],  # 이 예제에서는 추가 tool 불필요
    checkpointer=InMemorySaver(),  # 대화 상태 관리용
    pre_model_hook=pre_model_hook,  # LLM 호출 전 사용자 메시지 저장
    post_model_hook=post_model_hook,  # LLM 호출 후 assistant 응답을 episodic 처리를 위해 저장
)

## Step 6: Agent Runtime 구성

사용자 및 세션에 대한 고유 식별자로 Agent를 구성해야 합니다. 이러한 ID는 메모리 구성 및 검색에 중요합니다.

### Graph Invoke Input
인수 `inputs`로 최신 사용자 메시지만 전달하면 됩니다. 다른 상태 변수도 포함할 수 있지만 간단한 `create_react_agent`의 경우 메시지만 필요합니다.

### LangGraph RuntimeConfig
LangGraph에서 config는 호출 시점에 필요한 속성(예: 사용자 ID 또는 세션 ID)을 포함하는 `RuntimeConfig`입니다. `AgentCoreMemorySaver`의 경우 config에 `thread_id`와 `actor_id`를 설정해야 합니다. 예를 들어, AgentCore 호출 엔드포인트는 호출자의 신원 또는 사용자 ID를 기반으로 이를 할당할 수 있습니다. [추가 문서는 여기에서](https://langchain-ai.github.io/langgraphjs/how-tos/configuration/) 확인할 수 있습니다.



In [ ]:
actor_id = "user-1"
config = {
    "configurable": {
        "thread_id": "session-1",  # 필수: AgentCore session_id로 매핑됨
        "actor_id": actor_id,  # 필수: AgentCore actor_id로 매핑됨
    }
}

## Step 7: Agent 테스트

음식 선호도에 대한 대화를 통해 영양 어시스턴트를 테스트해 봅시다. Agent는 향후 회상 및 패턴 분석을 위해 대화를 에피소드로 자동 캡처합니다.

In [ ]:
# Agent 출력을 실행 중에 보기 좋게 출력하는 helper 함수
def run_agent(query: str, config: RunnableConfig):
    printed_ids = set()
    events = graph.stream(
        {"messages": [{"role": "user", "content": query}]},
        config,
        stream_mode="values",  # 전체 state 값을 스트리밍
    )
    for event in events:
        if "messages" in event:
            for msg in event["messages"]:
                # 이미 출력한 메시지는 건너뛰기 (중복 방지)
                if id(msg) not in printed_ids:
                    msg.pretty_print()
                    printed_ids.add(id(msg))


prompt = """
Hey there! Im cooking one of my favorite meals tonight, salmon with rice and veggies (healthy). Has
great macros for my weightlifting competition that is coming up. What can I add to this dish to make it taste better
and also improve the protein and vitamins I get?
"""

run_agent(prompt, config)

### 무엇이 저장되었나요?
보시다시피, Model은 아직 이전 식사 계획 세션의 인사이트를 가지고 있지 않습니다.

pre/post model hook을 사용한 이 구현에서는 두 개의 메시지가 여기에 저장되었습니다. 사용자의 첫 번째 메시지와 AI Model의 응답이 모두 AgentCore Memory에 대화 이벤트로 저장되었습니다. 에피소드 및 reflection이 생성되는 데 몇 분이 걸릴 수 있으므로 첫 번째 시도에서 아무것도 찾지 못하면 몇 분 후에 다시 시도하세요.

이러한 메시지는 episodic strategy에 의해 처리되어 AgentCore 장기 메모리에 구조화된 에피소드 및 reflection을 생성했습니다. 실제로 store를 직접 확인하여 지금까지 저장된 내용을 확인할 수 있습니다:

In [ ]:
# 대화 메시지 검색
search_namespace = ("nutrition", actor_id, "session-1/")
result = store.search(search_namespace, query="meal", limit=3)
print(f"Conversation messages result: {result}")

In [ ]:
# LangGraph에서 episodic 장기 메모리를 검색하는 올바른 방법
from bedrock_agentcore.memory import MemoryClient

# Store가 아닌 memory client를 직접 사용
memory_client = MemoryClient(region_name=region)

print("=== Searching Long-Term Episodic Memories ===")
print(f"Memory ID: {memory_id}")
print()

# Episodic memory (episode) 검색
print("1. Episodic namespace: nutrition/user-1/session-1/")
try:
    episodes = memory_client.retrieve_memories(
        memory_id=memory_id,
        namespace="nutrition/user-1/session-1/",
        query="meal",
        top_k=3,
    )
    print(f"   Found {len(episodes)} episode memories")
    for mem in episodes:
        content = mem.get("content", {})
        text = content.get("text", str(content))
        print(f"   - {text[:300]}...")
except Exception as e:
    print(f"   Error: {e}")
print()

# Reflection memory 검색
print("2. Reflection namespace: nutrition/user-1/")
try:
    reflections = memory_client.retrieve_memories(
        memory_id=memory_id, namespace="nutrition/user-1/", query="meal", top_k=3
    )
    print(f"   Found {len(reflections)} reflection memories")
    for mem in reflections:
        content = mem.get("content", {})
        text = content.get("text", str(content))
        print(f"   - {text[:300]}...")
except Exception as e:
    print(f"   Error: {e}")

### Store에 대한 Agent 액세스

**참고** - AgentCore Memory가 백그라운드에서 이러한 이벤트를 처리하므로 메모리가 추출되고 장기 메모리 검색에 임베딩되는 데 몇 분이 걸릴 수 있습니다.

좋습니다! 이제 이전 대화의 메시지를 기반으로 장기 메모리가 namespace에 추출된 것을 확인했습니다.

이제 새 세션을 시작하고 저녁 식사로 무엇을 요리할지에 대한 추천을 요청해 봅시다. Agent는 store를 사용하여 추출된 장기 메모리에 액세스하여 사용자가 확실히 좋아할 추천을 할 수 있습니다.

In [ ]:
config = {
    "configurable": {
        "thread_id": "session-2",  # 새로운 세션 ID
        "actor_id": actor_id,  # 동일한 actor ID
    }
}

run_agent("Today's a new day, what should I make for dinner tonight?", config)

### 마무리

보시다시피, Agent의 대화는 자동으로 캡처되고 턴별 분석과 함께 구조화된 에피소드로 처리됩니다. episodic strategy는 여러 식사 계획 세션에 걸쳐 인사이트를 생성하여 패턴을 식별하고 선호도가 시간에 따라 어떻게 변화하는지 추적합니다.

AgentCoreMemoryStore는 매우 유연하며 pre/post model hook 또는 store 작업이 있는 Tool 자체를 포함하여 다양한 방식으로 구현할 수 있습니다. checkpointing을 위한 AgentCoreMemorySaver와 함께 사용하면 전체 대화 상태와 episodic reflection을 결합하여 복잡하고 지능적인 Agent 시스템을 구성할 수 있습니다.